In [9]:
# check xgboost version
import xgboost as xgb
print(xgb.__version__)
import warnings
warnings.filterwarnings('ignore')


1.7.1


In [2]:
import pandas as pd
import numpy as np


# For tuning the model

from sklearn.model_selection import RandomizedSearchCV

In [6]:
X_train = pd.read_csv('/Users/zoso/Documents/Courses and Certifications/MIT_machineLearning/Shinkansen Travel Experience/X_train.csv')
X_test = pd.read_csv('/Users/zoso/Documents/Courses and Certifications/MIT_machineLearning/Shinkansen Travel Experience/X_test.csv')
y_train = pd.read_csv('/Users/zoso/Documents/Courses and Certifications/MIT_machineLearning/Shinkansen Travel Experience/y_train.csv')

X_nulls_train = pd.read_csv('/Users/zoso/Documents/Courses and Certifications/MIT_machineLearning/Shinkansen Travel Experience/X_nulls_train.csv')

'''xgb.set_config(verbosity=0)

# Get current value of global configuration

# This is a dict containing all parameters in the global configuration,

# including 'verbosity'

config = xgb.get_config()
assert config['verbosity'] == 2'''


meta = .19
max depth = 9

Best: 0.953146 using {'subsample': 0.8999999999999999, 'reg_lambda': 0.1, 'reg_alpha': 0.8, 'min_child_weight': 3.499999999999999, 'max_depth': 9, 'gamma': 0.31000000000000005, 'eta': 0.17999999999999997, 'colsample_bytree': 0.8999999999999999}

Best: 0.954492 using {'tree_method': 'approx', 'subsample': 0.8999999999999999, 'reg_lambda': 0.0, 'reg_alpha': 0.31, 'min_child_weight': 2.9999999999999996, 'max_depth': 12, 'gamma': 0.67, 'eta': 0.18999999999999995, 'colsample_bytree': 0.7999999999999999, 'colsample_bylevel': 0.8999999999999999}

Best: 0.954789 using {'tree_method': 'hist', 'subsample': 0.8999999999999999, 'reg_lambda': 0.42, 'reg_alpha': 0.54, 'min_child_weight': 1.7999999999999998, 'max_depth': 9, 'gamma': 0.24000000000000002, 'eta': 0.15999999999999998, 'colsample_bytree': 0.7999999999999999, 'colsample_bylevel': 0.5}

Best: 0.955626 using {'tree_method': 'auto', 'subsample': 0.8999999999999999, 'sampling_method': 'uniform', 'reg_lambda': 0.62, 'reg_alpha': 0.77, 'min_child_weight': 0.7999999999999999, 'max_depth': 9, 'gamma': 0.91, 'eta': 0.19999999999999996, 'colsample_bytree': 0.8999999999999999, 'colsample_bylevel': 0.6}

Best: 0.956018 using {'tree_method': 'auto', 'subsample': 0.8999999999999999, 'sampling_method': 'uniform', 'reg_lambda': 0.81, 'reg_alpha': 0.88, 'num_parallel_tree': 8, 'min_child_weight': 2.7999999999999994, 'max_depth': 9, 'gamma': 0.66, 'eta': 0.19999999999999996, 'colsample_bytree': 0.7, 'colsample_bylevel': 0.7}

In [7]:
X_ID = X_train.copy()
X_ID_test = X_test.copy()
X_train = X_train.drop(columns=['ID'])
X_test = X_test.drop(columns=['ID'])

In [8]:
# make predictions using xgboost for classification
from xgboost import XGBClassifier

model = XGBClassifier()

model = XGBClassifier(tree_method = 'auto',subsample= 0.8999999999999999, sampling_method = 'uniform',reg_lambda=.81,reg_alpha=0.88,num_parallel_tree= 8,min_child_weight= 2.7999999999999994, max_depth= 9, gamma= 0.66, eta= 0.19999999999999996, colsample_bytree= 0.7,colsample_bylevel=0.7)
# fit the model on the whole dataset
model.fit(X_train, y_train)
# make a single prediction
yhat = model.predict(X_test)


In [ ]:
pred = np.array(yhat)
id = np.array(X_ID_test['ID'])
pred_y = pd.DataFrame([id,pred])
sumbmission =pred_y.T.rename(columns={0:'ID',1:'Overall_Experience'})
sumbmission.to_csv('/Users/zoso/Documents/Courses and Certifications/MIT_machineLearning/Shinkansen Travel Experience/Sumbmission_XGB.csv',index=False)
sumbmission

,ID,Overall_Experience
0,99900001,1
1,99900002,1
2,99900003,1
3,99900004,0
4,99900005,1
...,...,...
35597,99935598,0
35598,99935599,1
35599,99935600,0
35600,99935601,1


# Tuning

In [10]:
# define the grid search parameters
xgb = XGBClassifier()#(tree_method = 'approx',subsample= 0.8999999999999999, reg_lambda=0.001,reg_alpha=0.31,min_child_weight= 2.9999999999999996, gamma= 0.67, eta= 0.18999999999999995, colsample_bytree= 0.7999999999999999,colsample_bylevel=0.8999999999999999)
param_random = {
    'gamma':np.arange(.01,1,.01),
    'eta':np.arange(.1,.3,.01),
    'max_depth' : np.arange(2,10),
    'colsample_bytree':np.arange(.5,1,.1),
    'subsample':np.arange(.5,1,.1),
    'min_child_weight':np.arange(.5,5,.1),
    'reg_lambda':np.arange(0,1,.01),
    'reg_alpha':np.arange(0,1,.01),
    'colsample_bylevel':np.arange(.5,1,.1),
    'tree_method':['auto','exact','approx','hist','gpu_hist'],
    #'booster':['gbtree','dart'],
    #'max_delta_step' : np.arange(11)
    'sampling_method':['uniform','gradient_based'],
    #'scale_pos_weight' : np.arange(.5,1,.01),
    #'refresh_leaf':[0,1],
    'num_parallel_tree':np.arange(1,20),
    #'updater': ['grow_colmaker','grow_histmaker','grow_quantile_histmaker','grow_gpu_hist','sync','refresh','prune'],
    #'process_type':['deafult','update']
    }

kfold_splits = 10
random= RandomizedSearchCV(estimator=xgb,  
                    verbose=1,
                    cv=kfold_splits,  
                    param_distributions=param_random,n_jobs=-1)
                    
random_result = random.fit(X_train, y_train,verbose=1) 

# Summarize results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))
means = random_result.cv_results_['mean_test_score']
stds = random_result.cv_results_['std_test_score']
params = random_result.cv_results_['params']

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Exception ignored on calling ctypes callback function: <bound method DataIter._next_wrapper of <xgboost.data.SingleBatchInternalIter object at 0x7fcc309c4a30>>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/base2/lib/python3.9/site-packages/xgboost/core.py", line 500, in _next_wrapper
    def _next_wrapper(self, this: None) -> int:  # pylint: disable=unused-argument
KeyboardInterrupt: 


KeyboardInterrupt: 

Best: 0.955965 using {'tree_method': 'hist', 'subsample': 0.8999999999999999, 'sampling_method': 'uniform', 'reg_lambda': 0.47000000000000003, 'reg_alpha': 0.01, 'num_parallel_tree': 4, 'min_child_weight': 3.999999999999999, 'max_depth': 8, 'gamma': 0.45, 'eta': 0.22999999999999995, 'colsample_bytree': 0.8999999999999999, 'colsample_bylevel': 0.7}

Best: 0.956018 using {'tree_method': 'auto', 'subsample': 0.8999999999999999, 'sampling_method': 'uniform', 'reg_lambda': 0.81, 'reg_alpha': 0.88, 'num_parallel_tree': 8, 'min_child_weight': 2.7999999999999994, 'max_depth': 9, 'gamma': 0.66, 'eta': 0.19999999999999996, 'colsample_bytree': 0.7, 'colsample_bylevel': 0.7}

In [ ]:
# make predictions using xgboost for classification
from xgboost import XGBClassifier

model = XGBClassifier(tree_method = 'auto',subsample= 0.8999999999999999, sampling_method = 'uniform',reg_lambda=.81,reg_alpha=0.88,num_parallel_tree= 8,min_child_weight= 2.7999999999999994, max_depth= 9, gamma= 0.66, eta= 0.19999999999999996, colsample_bytree= 0.7,colsample_bylevel=0.7)
# fit the model on the whole dataset
model.fit(X_train, y_train)
# make a single prediction
yhat = model.predict(X_test)


In [ ]:
pred = np.array(yhat)
id = np.array(X_test['ID'])
pred_y = pd.DataFrame([id,pred])
sumbmission =pred_y.T.rename(columns={0:'ID',1:'Overall_Experience'})
sumbmission.to_csv('/Users/zoso/Documents/Courses and Certifications/MIT_machineLearning/Shinkansen Travel Experience/Sumbmission_XGB_tuned.csv',index=False)
sumbmission

,ID,Overall_Experience
0,99900001,1
1,99900002,1
2,99900003,1
3,99900004,0
4,99900005,1
...,...,...
35597,99935598,0
35598,99935599,1
35599,99935600,0
35600,99935601,1
